In [ ]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from tqdm import tqdm

In [ ]:
input_size=28
sequence_length =28
num_layers=2
hidden_size=256

learning_rate = 0.001
num_epochs = 5

num_classes =10
batch_size = 64

In [ ]:
class SimpleRNN(nn.Module):
    def __init__(self, input_size, num_layers, hidden_size, sequence_length, num_classes):
        super(SimpleRNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size= hidden_size

        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size * sequence_length, num_classes)

    def forward(self, x):

        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        out, _ = self.rnn(x, h0)
#         print(out.shape)
        out = out.reshape(out.shape[0], -1)
        out = self.fc1(out)
        return out

In [ ]:
class SimpleGRU(nn.Module):
    def __init__(self, input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, num_classes=num_classes, sequence_length=sequence_length):
        super(SimpleGRU, self).__init__()
        self.hidden_size  = hidden_size
        self.num_layers = num_layers

        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size * sequence_length, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        out,_ = self.gru(x, h0)
        out = out.reshape(out.shape[0], -1)
        out = self.fc1(out)
        return out


In [ ]:
class SimpleLSTM(nn.Module):
    def __init__(self, input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, sequence_length=sequence_length, num_classes=num_classes):
        super(SimpleLSTM, self).__init__()

        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size * sequence_length, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device=device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device=device)

        out, _ = self.lstm(x,(h0, c0))
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        return out

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
models = [SimpleRNN( input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, num_classes=num_classes, sequence_length=sequence_length).to(device=device),SimpleGRU().to(device=device),SimpleLSTM().to(device=device)]

In [ ]:
for model in models:
  print(model._get_name())
  x = torch.randn(64,28,28).to(device=device)
  y = model(x)
  print(y.shape)

SimpleRNN
torch.Size([64, 10])
SimpleGRU
torch.Size([64, 10])
SimpleLSTM
torch.Size([64, 10])


In [ ]:
import pandas as pd
import numpy as np

class MnistDataset(Dataset):
    def __init__(self, datapath):
        super(MnistDataset).__init__()
        df = pd.read_csv(datapath, dtype=np.float)

        self.x = torch.from_numpy(df.iloc[:, 1:].values)
        self.x = self.x.reshape(self.x.size(0), 1, 28, 28).squeeze(1) # GRU and RNN expect N * 28 * 28
        self.x = self.x.float()

        self.y = torch.from_numpy(df.iloc[:, 0].values)
        self.y = self.y.long()

        self.n_samples = df.shape[0]

    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return self.n_samples

In [ ]:
transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
])


In [ ]:
train_dataset = datasets.MNIST(root='./', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./', train=False, transform=transform, download=True)

In [ ]:
x, y = train_dataset[0]
print(x.shape, y)

torch.Size([1, 28, 28]) 5


In [ ]:
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
loss_criterion  = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
for model in models:
  print(model._get_name())
  current_loss = 0
  for epoch in range(num_epochs):
      for data, target in tqdm(train_dataloader):
          data = data.to(device=device)
          target = target.to(device=device)
          data.squeeze_(1)
          score = model(data)
          loss = loss_criterion(score, target)
          current_loss = loss

          optimizer.zero_grad()
          loss.backward()

          optimizer.step()
      print(f"At epoch: {epoch}, loss: {current_loss}")

SimpleRNN


100%|██████████| 938/938 [00:11<00:00, 79.41it/s] 


At epoch: 0, loss: 2.286947250366211


100%|██████████| 938/938 [00:08<00:00, 109.71it/s]


At epoch: 1, loss: 2.3030507564544678


...

At epoch: 2, loss: 0.005351718980818987


100%|██████████| 938/938 [00:10<00:00, 87.57it/s]


At epoch: 3, loss: 0.0028403163887560368


100%|██████████| 938/938 [00:10<00:00, 85.37it/s]

At epoch: 4, loss: 0.00018620127229951322


In [ ]:
def check_accuracy(dlr,model):

    total_correct = 0
    total_samples = 0

    model.eval()

    with torch.no_grad():
        for x, y in dlr:
            x = x.to(device=device)
            y = y.to(device=device)
            x.squeeze_(1)
            score = model(x)
            _,predictions = score.max(1)

            total_correct += (y==predictions).sum()
            total_samples += predictions.size(0)

    model.train()
    print(f"total samples: {total_samples} total_correct: {total_correct} accuracy : {float(total_correct/total_samples)* 100}")

In [ ]:
for model in models:
  print(model._get_name())
  check_accuracy(train_dataloader, model)
  check_accuracy(test_dataloader, model)

SimpleRNN
total samples: 60000 total_correct: 6153 accuracy : 10.254999995231628
total samples: 10000 total_correct: 1058 accuracy : 10.579999536275864
SimpleGRU
total samples: 60000 total_correct: 7308 accuracy : 12.1799997985363
total samples: 10000 total_correct: 1159 accuracy : 11.589999496936798
SimpleLSTM
total samples: 60000 total_correct: 59470 accuracy : 99.11666512489319
total samples: 10000 total_correct: 9872 accuracy : 98.71999621391296
